# Twitter Example
In order to use all of this though, we need to setup a Developer API acocunt with Twitter and create an application to get credentials. The link below helps you to create the Developer API. 

    https://apps.twitter.com/
    
Once we are done with the App Creation, we also need to install python-twitter, a python library to connect your Python to the twitter dev account.

Let's get started!


#### <span style="color:Red">Points worth Noting before running this</span>
1. Begin by running the TweetRead.py file. Make sure to add your own IP Adress and your credential keys.
2. You can only configure SparkContext one time. If you configure it again it will yield nothing but an error. The appropriate reason is that a notebook session can only configure single SparkContext session in one go.
3. To run this code, your machine should configure with:
    - Spark Installed along with its Path variable initiaition.
    - Python version above 3 (can also be executed in version less than 3 but certainly needs some minor changes in the library changes for spark).

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
%%time

In [2]:
# Can only run this once. restart your kernel for any errors.
sc = SparkContext()
%%time

In [3]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)
%%time

In [4]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5342)
print("Now listening")
%%time

In [5]:
lines = socket_stream.window( 20 )
%%time

In [6]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )
%%time

In [7]:
# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10) ) ) # Registers to a table.

#### Now is the time to run the TweetRead.py (This actually starts the socket mapping and configure the connection)

#### Once we run the listener programme then is the time to start the spark streaming.

In [8]:
ssc.start()    

#### Once you are satisfied with your results and tweet generation. You can stop the streaming using:

In [9]:
ssc.stop()